##  <center> Quelques techniques pour optimiser une application Spark </center>

## 1. Broadcasting

Lorsque vous exécutez une application Spark, les jobs sont envoyés aux différents workers entrainant beaucoup d'échanges de données ces derniers et le driver. Par exemple :

```scala
val  rdd = sc.parallelize(Seq(1, 2, 3, 4))
val  const = 10
rdd.map(x => x + const).collect
```

Comme les données du RDD sont partitionnées, chaque exécuteur reçoit une copie de la variable `const` car celle-ci est définie sur le driver. Cela peut provoquer des problèmes de performances notamment lors d'une jointure. La technique d'un broadcasting qui permet d'optimiser ce type de problème en rendant disponible une variable (par exemple `const ` dans l'exemple) à tous les nœuds. 
Remarques : 

* Le broadcasting est contrôlée par le paramètre de configuration `spark.sql.autoBroadcastJoinThreshold`, dont la valeur par défaut est de 10 Mo.
* Une variable broadcastée n'est pas modifiable (read-only variable). Cela permet de s'assurer que tous les workers utilisent la même valeur.
* Pour les fonctions parallélisées utilisant en argument une donnée relativement importante, l'utilisation du broadcasting peut améliorer la performance.


In [2]:
import spark.implicits._

val simpleData = Seq(("James", "SLS", 3000),
    ("Michael", "SLS", 4600),
    ("Robert", "SLS", 4100),
    ("Maria", "FIN", 3000),
    ("James", "SLS", 3000),
    ("Scott", "FIN", 3300),
    ("Jen", "FIN", 3900),
    ("Jeff", "MKT", 3000),
    ("Kumar", "MKT", 2000),
    ("Saif", "SLS", 4100)
  )

val employeDF = simpleData.toDF("EMP_NAME", "DEP_CD", "SALARY")
employeDF.show()

val depDF = Seq(
  ("SLS", "Sales"),
  ("FIN", "Finance"),
  ("MKT", "Marketing")
).toDF("DEP_CD", "DEP_LIB")

val deptMap = Map("SLS" -> "Sales",
                    "FIN" -> "Finance",
                    "MKT" -> "Marketing")

import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.broadcast.Broadcast

def mapping(maps: Broadcast[Map[String, String]]) : String => String = {x => maps.value.getOrElse(x, "No. Dep")}
val func = mapping(sc.broadcast(deptMap))
val transformer = udf(func)

employeDF.withColumn("DEP_CD", transformer(col("DEP_CD"))).show()

+--------+------+------+
|EMP_NAME|DEP_CD|SALARY|
+--------+------+------+
|   James|   SLS|  3000|
| Michael|   SLS|  4600|
|  Robert|   SLS|  4100|
|   Maria|   FIN|  3000|
|   James|   SLS|  3000|
|   Scott|   FIN|  3300|
|     Jen|   FIN|  3900|
|    Jeff|   MKT|  3000|
|   Kumar|   MKT|  2000|
|    Saif|   SLS|  4100|
+--------+------+------+

+--------+---------+------+
|EMP_NAME|   DEP_CD|SALARY|
+--------+---------+------+
|   James|    Sales|  3000|
| Michael|    Sales|  4600|
|  Robert|    Sales|  4100|
|   Maria|  Finance|  3000|
|   James|    Sales|  3000|
|   Scott|  Finance|  3300|
|     Jen|  Finance|  3900|
|    Jeff|Marketing|  3000|
|   Kumar|Marketing|  2000|
|    Saif|    Sales|  4100|
+--------+---------+------+



import spark.implicits._
simpleData: Seq[(String, String, Int)] = List((James,SLS,3000), (Michael,SLS,4600), (Robert,SLS,4100), (Maria,FIN,3000), (James,SLS,3000), (Scott,FIN,3300), (Jen,FIN,3900), (Jeff,MKT,3000), (Kumar,MKT,2000), (Saif,SLS,4100))
employeDF: org.apache.spark.sql.DataFrame = [EMP_NAME: string, DEP_CD: string ... 1 more field]
depDF: org.apache.spark.sql.DataFrame = [DEP_CD: string, DEP_LIB: string]
deptMap: scala.collection.immutable.Map[String,String] = Map(SLS -> Sales, FIN -> Finance, MKT -> Marketing)
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.broadcast.Broadcast
mapping: (maps: org.apache.spark.broadcast.Broadcast[Map[String,String]])String => String
func: String => String = $Lambda$2528/1686643672@1f171d23
transformer: org.a...


## 2. Accumulateurs

Un accumulateur est une variable globale donc modifiable par les workers. Il n'accepte que des opérations associatives ou commutatives.

Les workers peuvent modifier la valeur d'un accumulateur mais seul le driver peut lire sa valeur.

In [3]:
val accum = sc.longAccumulator("Exemple accumulateur")

sc.parallelize(Array(1, 2, 3, 4)).foreach(x => accum.add(x))
 
accum.value

accum: org.apache.spark.util.LongAccumulator = LongAccumulator(id: 5, name: Some(Exemple accumulateur), value: 10)
res2: Long = 10


## 3. Quelles fonctions à éviter quand c'est possible

* `collect` : envoie toutes les données vers le driver       

* `count` : cette fonction prend beaucoup de temps

* `groupByKey` Il est préférable d'utiliser `reduceByKey` au lieu de `groupByKey` pour avoir une meilleure performance en termes de temps de calcul et de mémoire utilisé. Pour avoir plus d'options, vous pouvez utiliser `combineByKey` (`reduceByKey` est implémenté en utilisant `combineByKey`)

* Les fonctions `udf` peuvent prendre beaucoup car elles ne sont pas souvent bien optimisées. Donc elles sont à éviter quand c'est possible surtout pour les utilisateurs de PySpark


## 4. Cache et Persistence

Lorsqu’on accède à un dataframe de façon itérative, utiliser la technique du caching peut améliorer les performances. Pour mettre en cache des données, Spark fournit deux fonctions :

* `cache` : stocke la donnée en mémoire
* `persist` : stocke la donnée en mémoire et / ou sur le disque
* `cache = persit(StorageLevel.MEMORY_ONLY)`

Ce n'est pas toujours pertinent de mettre en cache des données. Lorsqu'on lit une fois des données, il n'est pas souvent nécessaire de mettre en cache car cela ne peut ralentir votre travail.

## 5. Sérialisation des données

Le calcul distribué de Spark entraine d'importantes étapes de sérialisation de données. En fonction du format utilisé, ces étapes peuvent très coûteuses en temps. Par défaut, Spark utilise la sérialisation Java. Mais vous pouvez utiliser sérialisation Kryo (parfois 10 fois rapide).

`conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")`

## 6. Répartition des  Données

Le re-partitionnement des données est une piste pour améliorer les performances d'une application Spark.

* Eviter les partitions trop petites : cela peut entrainer de beaucoup de petites tâches à effectuer. Vous pouvez répartir à nouveau les données en utilisant :
    * `repartition(n)` 
    * ou `coalesce(n, shuffle=true)`
* Eviter les partitions déséquilibrées

**Sources :**    

[Documentation Spark](https://spark.apache.org/docs/latest/sql-performance-tuning.html)     
[Documentation Microsoft](https://docs.microsoft.com/fr-fr/azure/hdinsight/spark/apache-spark-overview)   
[Apache Spark Optimisation Techniques and Performance Tuning](https://www.xenonstack.com/blog/apache-spark-optimisation/)    
[Apache Spark - Best practices and tuning](https://umbertogriffo.gitbook.io/apache-spark-best-practices-and-tuning/)